In [1]:
!pip install datasets transformers accelerate accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.3/506.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 18.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.10.1
    Uninstalling accelerate-1.10.1:
      Successfully uninstalled accelerate-1.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you

In [2]:
from transformers import BertTokenizerFast, BertForSequenceClassification
from datasets import load_dataset
from transformers import Trainer, TrainingArguments

In [3]:
emotion= load_dataset('emotion')

README.md: 0.00B [00:00, ?B/s]

split/train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

split/validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

split/test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [4]:
emotion

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [5]:
tokenizer= BertTokenizerFast.from_pretrained('bert-base-uncased')
model= BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
text1='Hai, How are you? , what are you doing?'
text2= 'I am fine, How about you?'
inputs=[text1,text2]

In [7]:
tokenizer.tokenize(text1)

['hai', ',', 'how', 'are', 'you', '?', ',', 'what', 'are', 'you', 'doing', '?']

In [8]:
tokenizer.tokenize(inputs)

['hai',
 ',',
 'how',
 'are',
 'you',
 '?',
 ',',
 'what',
 'are',
 'you',
 'doing',
 '?',
 'i',
 'am',
 'fine',
 ',',
 'how',
 'about',
 'you',
 '?']

In [12]:
output=tokenizer(text1,text2)
output

{'input_ids': [101, 15030, 1010, 2129, 2024, 2017, 1029, 1010, 2054, 2024, 2017, 2725, 1029, 102, 1045, 2572, 2986, 1010, 2129, 2055, 2017, 1029, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [24]:
tokenizer.decode(output['input_ids'])

'[CLS] hai, how are you?, what are you doing? [SEP] i am fine, how about you? [SEP]'

In [25]:
def pre_process(example):
  return tokenizer(example['text'],padding=True,truncation=True,max_length=128)

In [26]:
emotion_emb= emotion.map(pre_process,batched=True)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

In [27]:
emotion_emb

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [31]:
train_data= emotion_emb['train'].shuffle(seed=40).select(range(32))
val_data= emotion_emb['train'].shuffle(seed=40).select(range(16))

In [32]:
train_data= train_data.remove_columns('text')
val_data= val_data.remove_columns('text')

In [33]:
train_data

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 32
})

In [35]:
train_data.set_format('torch')
val_data.set_format('torch')

In [38]:
train_args= TrainingArguments(output_dir='results',
                              learning_rate=2e-5,
                              weight_decay=0.01,
                              per_device_train_batch_size=4,
                              per_device_eval_batch_size=4,
                              eval_strategy='epoch',
                              num_train_epochs=1
                              )

In [40]:
trainer= Trainer(args=train_args,model=model,train_dataset=train_data,eval_dataset=val_data)

In [ ]:
print('start train')
trainer.train()
print('summary result')

start train


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

In [ ]:
pred= trainer.predict(val_data)
pred1= pred.predictions.argmax(axis=1)
val_ids= pred.label_ids

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(val_ids,pred1)